In [41]:
import xarray as xr 
from preprocessing.utils import scaleLongitudes
import geopandas as gpd
import pandas as pd

#convert a list of tuples into a pandas dataframe
def convertToDF(list_of_tuples):
    df = pd.DataFrame(list_of_tuples, columns = ['lat','lon']) 
    return df

def getEcoZoneCoordinates():
    ecozones = gpd.read_file('data/shapefiles/ecozones.shp').to_crs('epsg:4326')
    netcdf_file = xr.open_dataset(f'data/CESM/treeFrac_Lmon_CESM2_historical_r11i1p1f1_gn_199901-201412.nc')
    netcdf_file = scaleLongitudes(netcdf_file)
    dissolved_ecozones = ecozones.dissolve(by='ZONE_NAME').reset_index()
    netcdf_file = netcdf_file['treeFrac']
    df = pd.DataFrame()
    for _,region in dissolved_ecozones.iterrows():
        gdf = gpd.GeoDataFrame(geometry=[region.geometry])
        netcdf_file.rio.set_spatial_dims(x_dim="lon", y_dim="lat", inplace=True)
        netcdf_file.rio.write_crs("epsg:4326", inplace=True)
        clipped = netcdf_file.rio.clip(gdf.geometry, ecozones.crs,drop=True)
        stacked = clipped.stack(x=['lat','lon'])
        # stacked.isel(time=0)[stacked.isel(time=0).notnull()].coords['x'].values
        val = stacked.isel(time=0)[stacked.isel(time=0).notnull()].coords['x'].values
        x = convertToDF(list(val))
        x['zone'] = region['ZONE_NAME']
        df= pd.concat([df,x])
    df['lat'] = df['lat'].round(7)
    return df

In [52]:
from preprocessing.utils import getArea
from hydra import initialize, compose

with initialize(version_base=None, config_path="conf"):
    cfg = compose(config_name="config")
df = getEcoZoneCoordinates()
t = 0
nfis_agb = pd.read_csv('data/nfis_agb.csv')
for region in df.zone.unique():
    print(region)
    total_agb = pd.merge(nfis_agb,df[df['zone'] == region],on=['lat','lon']) 
    if(len(total_agb) == 0):
        continue
    total_agb['agb'] = total_agb['agb'] / 10
    total_agb['area'] = total_agb.apply(lambda x: getArea(x['lat'],x['lon'],cfg),axis=1)
    total_agb['agb'] = total_agb['agb'] * total_agb['area']
    t = t + total_agb['agb'].sum() / 1e12
    print(region,total_agb['agb'].sum() / 1e9)
print(t)

/Users/gclyne/thesis/env/lib/python3.9/site-packages/xarray/conventions.py:521: SerializationWarning: variable 'treeFrac' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(


Arctic Cordillera
Atlantic Maritime
Atlantic Maritime 920.0439550624624
Boreal Cordillera
Boreal Cordillera 1628.469533485987
Boreal PLain
Boreal PLain 3964.1256520090856
Boreal Shield
Boreal Shield 6894.267281492343
Hudson Plain
Hudson Plain 29.269150732411973
MixedWood Plain
MixedWood Plain 284.6167856669681
Montane Cordillera
Montane Cordillera 3712.192844689875
Northern Arctic
Pacific Maritime
Pacific Maritime 1907.061142629115
Prairie
Prairie 200.97807787729695
Southern Arctic
Taiga Cordillera
Taiga Cordillera 23.054264406683057
Taiga Plain
Taiga Plain 1244.8158155033857
Taiga Shield
Taiga Shield 153.00213673688546
20.961896640292498
